In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers
import datetime

In [2]:
#define constants
EPOCHS = 10
BATCH_SIZE = 128
VERBOSE = 1
OPTIMIZER = tf.keras.optimizers.Adam()
VALIDATION_SPLIT = 0.95

In [3]:
#image characteristics
IMG_ROWS, IMG_COLS = 28, 28 #input image dimensions
INPUT_SHAPE = (IMG_ROWS, IMG_COLS, 1)
NB_CLASSES = 10 #number of outputs (10 digits)

In [4]:
#define the CNN (LeNet network proposed by Yann LeCun)
def build(input_shape, classes):
    model = models.Sequential()
    #CONV --> RELU --> POOL
    model.add(layers.Conv2D(20, (5,5), activation = 'relu', input_shape = input_shape))
    model.add(layers.MaxPooling2D(pool_size = (2,2), strides = (2,2)))
    #CONV --> RELU --> POOL
    model.add(layers.Conv2D(50, (5,5), activation = 'relu'))
    model.add(layers.MaxPooling2D(pool_size = (2,2), strides = (2,2)))
    #FLATTEN --> RELU
    model.add(layers.Flatten())
    model.add(layers.Dense(500, activation = 'relu'))
    #SOFTMAX classifier
    model.add(layers.Dense(classes, activation = "softmax"))
    return model

In [5]:
#load data
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()
#reshape data
X_train = X_train.reshape((60000, 28, 28, 1))
X_test = X_test.reshape((10000, 28, 28, 1))
#Normalization within [0,1]
X_train = X_train / 255.0
X_test = X_test / 255.0
#cast
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')


In [6]:
#convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, NB_CLASSES)
y_test = tf.keras.utils.to_categorical(y_test, NB_CLASSES)

In [7]:
#initialize the optimizer and the model
model = build(input_shape = INPUT_SHAPE, classes = NB_CLASSES)
model.compile(loss = "categorical_crossentropy", optimizer = OPTIMIZER, metrics = ["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 20)        520       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 20)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 50)          25050     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 50)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 500)               400500    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5

In [8]:
#TensorBoard
LOG_DIR = "logs/CNN_LeNet_mnist/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=1)
callbacks = [tf.keras.callbacks.TensorBoard(log_dir = LOG_DIR)]

In [9]:
#fit the model and evaluate
history = model.fit(X_train, y_train, batch_size = BATCH_SIZE, epochs = EPOCHS, verbose = VERBOSE, validation_split = VALIDATION_SPLIT, callbacks = callbacks)
score = model.evaluate(X_test, y_test, verbose = VERBOSE)
print("\nTest score:", score[0])
print("Test accuracy:", score[1])

Epoch 1/10
 1/24 [>.............................] - ETA: 0s - loss: 2.2940 - accuracy: 0.1562WARNING:tensorflow:From C:\Users\ER180124\.conda\envs\PrognosticEnv\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
24/24 [==============================] - 11s 454ms/step - loss: 1.2066 - accuracy: 0.6617 - val_loss: 0.6323 - val_accuracy: 0.7995
Epoch 2/10
24/24 [==============================] - 9s 390ms/step - loss: 0.3756 - accuracy: 0.8827 - val_loss: 0.2950 - val_accuracy: 0.9093
Epoch 3/10
24/24 [==============================] - 11s 449ms/step - loss: 0.2256 - accuracy: 0.9337 - val_loss: 0.2258 - val_accuracy: 0.9299
Epoch 4/10
24/24 [==============================] - 11s 438ms/step - loss: 0.1580 - accuracy: 0.9540 - val_loss: 0.2014 - val_accuracy: 0.9368
Epoch 5/10
24/24 [============================

In [10]:
%load_ext tensorboard

In [11]:
%tensorboard --logdir LOG_DIR

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 14196.